<a href="https://colab.research.google.com/github/jmbanda/BigDataProgramming_2019/blob/master/Class18_BasicMachineLearning_ClassificationandClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Machine Learning - Classification and Clustering

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

## Gaussian Naive Bayes

Perhaps the easiest naive Bayes classifier to understand is Gaussian naive Bayes.
In this classifier, the assumption is that *data from each label is drawn from a simple Gaussian distribution*.
Imagine that you have the following data:

In [0]:
from sklearn.datasets import make_blobs
X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu');

This procedure is implemented in Scikit-Learn's ``sklearn.naive_bayes.GaussianNB`` estimator:

In [0]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X, y);

Now let's generate some new data and predict the label:

In [0]:
rng = np.random.RandomState(0)
Xnew = [-6, -14] + [14, 18] * rng.rand(2000, 2)
ynew = model.predict(Xnew)

Now we can plot this new data to get an idea of where the decision boundary is:

In [0]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')
lim = plt.axis()
plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='RdBu', alpha=0.1)
plt.axis(lim);

We see a slightly curved boundary in the classifications—in general, the boundary in Gaussian naive Bayes is quadratic.

A nice piece of this Bayesian formalism is that it naturally allows for probabilistic classification, which we can compute using the ``predict_proba`` method:

In [0]:
yprob = model.predict_proba(Xnew)
yprob[-8:].round(2)

The columns give the posterior probabilities of the first and second label, respectively.
If you are looking for estimates of uncertainty in your classification, Bayesian approaches like this can be a useful approach.

Of course, the final classification will only be as good as the model assumptions that lead to it, which is why Gaussian naive Bayes often does not produce very good results.
Still, in many cases—especially as the number of features becomes large—this assumption is not detrimental enough to prevent Gaussian naive Bayes from being a useful method.

## Multinomial Naive Bayes

The Gaussian assumption just described is by no means the only simple assumption that could be used to specify the generative distribution for each label.
Another useful example is multinomial naive Bayes, where the features are assumed to be generated from a simple multinomial distribution.
The multinomial distribution describes the probability of observing counts among a number of categories, and thus multinomial naive Bayes is most appropriate for features that represent counts or count rates.

The idea is precisely the same as before, except that instead of modeling the data distribution with the best-fit Gaussian, we model the data distribuiton with a best-fit multinomial distribution.

### Example: Classifying Text

One place where multinomial naive Bayes is often used is in text classification, where the features are related to word counts or frequencies within the documents to be classified.
We discussed the extraction of such features from text in [Feature Engineering](05.04-Feature-Engineering.ipynb); here we will use the sparse word count features from the 20 Newsgroups corpus to show how we might classify these short documents into categories.

Let's download the data and take a look at the target names:

In [0]:
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups()
data.target_names

For simplicity here, we will select just a few of these categories, and download the training and testing set:

In [0]:
categories = ['talk.religion.misc', 'soc.religion.christian',
              'sci.space', 'comp.graphics']
train = fetch_20newsgroups(subset='train', categories=categories)
test = fetch_20newsgroups(subset='test', categories=categories)

Here is a representative entry from the data:

In [0]:
print(train.data[5])

In order to use this data for machine learning, we need to be able to convert the content of each string into a vector of numbers.
For this we will use the TF-IDF (term frequency–inverse document frequency) vectorizer , and create a pipeline that attaches it to a multinomial naive Bayes classifier:

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [0]:
model.fit(train.data, train.target)
labels = model.predict(test.data)

Now that we have predicted the labels for the test data, we can evaluate them to learn about the performance of the estimator.
For example, here is the confusion matrix between the true and predicted labels for the test data:

In [0]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(test.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=train.target_names, yticklabels=train.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

Evidently, even this very simple classifier can successfully separate space talk from computer talk, but it gets confused between talk about religion and talk about Christianity.
This is perhaps an expected area of confusion!

The very cool thing here is that we now have the tools to determine the category for *any* string, using the ``predict()`` method of this pipeline.
Here's a quick utility function that will return the prediction for a single string:

In [0]:
def predict_category(s, train=train, model=model):
    pred = model.predict([s])
    return train.target_names[pred[0]]

In [0]:
predict_category('sending a payload to the ISS')

In [0]:
predict_category('discussing islam vs atheism')

In [0]:
predict_category('determining the screen resolution')

Remember that this is nothing more sophisticated than a simple probability model for the (weighted) frequency of each word in the string; nevertheless, the result is striking.
Even a very naive algorithm, when used carefully and trained on a large set of high-dimensional data, can be surprisingly effective.

# Decision Trees

## Creating a decision tree

Consider the following two-dimensional data, which has one of four class labels:

In [0]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');

A simple decision tree built on this data will iteratively split the data along one or the other axis according to some quantitative criterion, and at each level assign the label of the new region according to a majority vote of points within it.
This figure presents a visualization of the first four levels of a decision tree classifier for this data:

This process of fitting a decision tree to our data can be done in Scikit-Learn with the ``DecisionTreeClassifier`` estimator:

In [0]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X, y)

Let's write a quick utility function to help us visualize the output of the classifier:

In [0]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()
    
    # Plot the training points
    ax.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=cmap,
               clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                         np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3,
                           levels=np.arange(n_classes + 1) - 0.5,
                           cmap=cmap,
                           zorder=1)

    ax.set(xlim=xlim, ylim=ylim)

Now we can examine what the decision tree classification looks like:

In [0]:
visualize_classifier(DecisionTreeClassifier(), X, y)

## Ensembles of Estimators: Random Forests

This notion—that multiple overfitting estimators can be combined to reduce the effect of this overfitting—is what underlies an ensemble method called *bagging*.
Bagging makes use of an ensemble (a grab bag, perhaps) of parallel estimators, each of which over-fits the data, and averages the results to find a better classification.
An ensemble of randomized decision trees is known as a *random forest*.

This type of bagging classification can be done manually using Scikit-Learn's ``BaggingClassifier`` meta-estimator, as shown here:

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8,
                        random_state=1)

bag.fit(X, y)
visualize_classifier(bag, X, y)

In this example, we have randomized the data by fitting each estimator with a random subset of 80% of the training points.
In practice, decision trees are more effectively randomized by injecting some stochasticity in how the splits are chosen: this way all the data contributes to the fit each time, but the results of the fit still have the desired randomness.
For example, when determining which feature to split on, the randomized tree might select from among the top several features.
You can read more technical details about these randomization strategies in the [Scikit-Learn documentation](http://scikit-learn.org/stable/modules/ensemble.html#forest) and references within.

In Scikit-Learn, such an optimized ensemble of randomized decision trees is implemented in the ``RandomForestClassifier`` estimator, which takes care of all the randomization automatically.
All you need to do is select a number of estimators, and it will very quickly (in parallel, if desired) fit the ensemble of trees:

In [0]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
visualize_classifier(model, X, y);

We see that by averaging over 100 randomly perturbed models, we end up with an overall model that is much closer to our intuition about how the parameter space should be split.

## Example: Random Forest for Classifying Digits

Earlier we took a quick look at the hand-written digits data.
Let's use that again here to see how the random forest classifier can be used in this context.

In [0]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.keys()

To remind us what we're looking at, we'll visualize the first few data points:

In [0]:
# set up the figure
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

# plot the digits: each image is 8x8 pixels
for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(digits.images[i], cmap=plt.cm.binary, interpolation='nearest')
    
    # label the image with the target value
    ax.text(0, 7, str(digits.target[i]))

We can quickly classify the digits using a random forest as follows:

In [0]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target,
                                                random_state=0)
model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

We can take a look at the classification report for this classifier:

In [0]:
from sklearn import metrics
print(metrics.classification_report(ypred, ytest))

And for good measure, plot the confusion matrix:

In [0]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, ypred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label');

We find that a simple, untuned random forest results in a very accurate classification of the digits data.

# Clustering

## Looking for clusters visually

You are given an array `points` of size 300x2, where each row gives the (x, y) co-ordinates of a point on a map.  Make a scatter plot of these points, and use the scatter plot to guess how many clusters there are.

Colab only code:

In [0]:
from google.colab import files
files.upload()

End of colab only code

**Step 1:** Load the dataset clustering1.csv *(found in iCollege)*.

In [0]:
import pandas as pd

df = pd.read_csv('clustering1.csv')
points = df.values

**Step 2:** Import PyPlot

In [0]:
import matplotlib.pyplot as plt

**Step 3:** Create an array called `xs` that contains the values of `points[:,0]` - that is, column `0` of `points`:

In [0]:
xs = points[:,0]

**Step 3:** Create an array called `ys` that contains the values of `points[:,1]` - that is, column `1` of `points`

In [0]:
ys = points[:,1]

**Step 4:** Make a scatter plot by passing `xs` and `ys` to the `plt.scatter()` function.

In [0]:
plt.scatter(xs, ys)

So what can we see here? How many clusters do we have?

## Clustering 2D points

From the scatter plot of the previous section, you saw that the points seem to separate into 3 clusters.  Now create a KMeans model to find 3 clusters, and fit it to the data points from the previous exercise.  After the model has been fit, obtain the cluster labels for points, and also for some new points using the `.predict()` method.

You are given the array `points` from the previous exercise, and also an array `new_points`.

**Step 1:** Load the dataset (clustering1.csv and clustering2.csv).

In [0]:
import pandas as pd

df = pd.read_csv('clustering1.csv')
points = df.values

new_df = pd.read_csv('clustering2.csv')
new_points = new_df.values

**Step 2:** Import `KMeans` from `sklearn.cluster`

In [0]:
from sklearn.cluster import KMeans

**Step 3:** Using `KMeans()`, create a `KMeans` instance called `model` to find `3` clusters. To specify the number of clusters, use the `n_clusters` keyword argument

In [0]:
model = KMeans(n_clusters=3)

**Step 4:** Use the `.fit()` method of `model` to fit the model to the array of points `points`.

In [45]:
model.fit(points)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

**Step 5:** Use the `.predict()` method of `model` to predict the cluster labels of `points`, assigning the result to `labels`.

In [0]:
labels = model.predict(points)

**Step 6:** Print out the labels, and have a look at them!  _(In the next section, I'll show you how to visualise this clustering better.)_

In [0]:
print(labels)

**Step 7:** Use the `.predict()` method of `model` to predict the cluster labels of `new_points`, assigning the result to `new_labels`.  Notice that KMeans can assign previously unseen points to the clusters it has already found!

In [0]:
new_labels = model.predict(new_points)

## Inspect your clustering

Let's now inspect the clustering you performed in the previous exercise!

**Step 1:** Load the dataset (clustering1.csv).

In [0]:
import pandas as pd

df = pd.read_csv('clustering1.csv')
points = df.values

**Step 2:** Run your solution to the previous section.

In [0]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3)
model.fit(points)
labels = model.predict(points)

**Step 3:** Import `matplotlib.pyplot` as `plt`

In [0]:
import matplotlib.pyplot as plt

**Step 4:** Assign column `0` of `points` to `xs`, and column `1` of `points` to `ys`

In [0]:
xs = points[:,0]
ys = points[:,1]

**Step 5:** Make a scatter plot of `xs` and `ys`, specifying the `c=labels` keyword arguments to color the points by their cluster label.  You'll see that KMeans has done a good job of identifying the clusters!

In [0]:
plt.scatter(xs, ys, c=labels)
plt.show()

**This is great**, but let's go one step further, and add the cluster centres (the "centroids") to the scatter plot.

**Step 6:** Obtain the coordinates of the centroids using the `.cluster_centers_` attribute of `model`.  Assign them to `centroids`.

In [0]:
centroids = model.cluster_centers_

**Step 7:** Assign column `0` of `centroids` to `centroids_x`, and column `1` of `centroids` to `centroids_y`.

In [0]:
centroids_x = centroids[:,0]
centroids_y = centroids[:,1]

**Step 8:** In a single cell, create two scatter plots (this will show the two on top of one another).  Call `plt.show()` just once, at the end.

Firstly, the make the scatter plot you made above.  Secondly, make a scatter plot of `centroids_x` and `centroids_y`, using `'X'` (a cross) as a marker by specifying the `marker` parameter. Set the size of the markers to be `200` using `s=200`.

In [0]:
plt.scatter(xs, ys, c=labels)
plt.scatter(centroids_x, centroids_y, marker='X', s=200)
plt.show()

**Great work!** The centroids are important because they are what enables KMeans to assign new, previously unseen points to the existing clusters.

## Example: How many clusters of grain?

You are given a dataset of the measurements of samples of grain.  What's a good number of clusters?

This dataset was obtained from the [UCI](https://archive.ics.uci.edu/ml/datasets/seeds).

**Step 1:** Load the dataset (seeds.csv).

In [0]:
import pandas as pd

seeds_df = pd.read_csv('seeds.csv')
# forget about the grain variety for the moment - we'll use this later
del seeds_df['grain_variety']

**Step 2:** Display the DataFrame to inspect the data.  Notice that there are 7 columns - so each grain sample (row) is a point in 7D space!  Scatter plots can't help us here.

In [0]:
seeds_df.head(10)

**Step 3:** Extract the measurements from the DataFrame using its `.values` attribute:

In [0]:
samples = seeds_df.values

**Step 4:**  Measure the quality of clusterings with different numbers of clusters using the
inertia.  For each of the given values of `k`, perform the following steps:

  - Create a `KMeans` instance called `model` with `k` clusters.
  - Fit the model to the grain data `samples`.
  - Append the value of the `inertia_` attribute of `model` to the list `inertias`.

In [0]:
from sklearn.cluster import KMeans

ks = range(1, 6)
inertias = []

for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k)

    # Fit model to samples
    model.fit(samples)

    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)

**Step 5:**  Plot the inertia to see which number of clusters is best. Remember: lower numbers are better!

In [0]:
import matplotlib.pyplot as plt

# Plot ks vs inertias
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

**Excellent work!** You can see from the graph that 3 is a good number of clusters, since these are points where the inertia begins to decrease more slowly.

## Exercise: Scaling fish data for clustering

You are given an array `samples` giving measurements of fish.  Each row represents asingle fish.  The measurements, such as weight in grams, length in centimeters, and the percentage ratio of height to length, have very different scales.  In order to cluster this data effectively, you'll need to standardize these features first.  In this exercise, you'll build a pipeline to standardize and cluster the data.

This great dataset was derived from the one [here](http://svitsrv25.epfl.ch/R-doc/library/rrcov/html/fish.html), where you can see a description of each measurement.

In [0]:
import pandas as pd

df = pd.read_csv('fish.csv')
species = list(df['species'])

# forget the species column for now - we'll use it later!
del df['species']

**Step 2:** Call `df.head()` to inspect the dataset:

In [0]:
df.head()

**Step 3:** Extract all the measurements as a 2D NumPy array, assigning to `samples` (hint: use the `.values` attribute of `df`)

In [0]:
samples = df.values

**Step 4:** Perform the necessary imports:

- `make_pipeline` from `sklearn.pipeline`.
- `StandardScaler` from `sklearn.preprocessing`.
- `KMeans` from `sklearn.cluster`.


In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

**Step 5:** Create an instance of `StandardScaler` called `scaler`.

In [0]:
scaler = StandardScaler()

**Step 6:** Create an instance of `KMeans` with `4` clusters called `kmeans`.

In [0]:
kmeans = KMeans(n_clusters=4)

**Step 7:** Create a pipeline called `pipeline` that chains `scaler` and `kmeans`. To do this, you just need to pass them in as arguments to `make_pipeline()`.

In [0]:
pipeline = make_pipeline(scaler, kmeans)

**Great job!** Now you're all set to transform the fish measurements and perform the clustering.  Let's get to it in the next exercise!

**Step 8:** Fit the pipeline to the fish measurements `samples`.

In [0]:
pipeline.fit(samples)

**Step 9:** Obtain the cluster labels for `samples` by using the `.predict()` method of `pipeline`, assigning the result to `labels`.

In [0]:
labels = pipeline.predict(samples)

**Step 10:** Using `pd.DataFrame()`, create a DataFrame `df` with two columns named `'labels'` and `'species'`, using `labels` and `species`, respectively, for the column values.

In [0]:
df = pd.DataFrame({'labels': labels, 'species': species})

**Step 11:** Using `pd.crosstab()`, create a cross-tabulation `ct` of `df['labels']` and `df['species']`.

In [0]:
ct = pd.crosstab(df['labels'], df['species'])

**Step 12:** Display your cross-tabulation, and check out how good your clustering is!

In [0]:
ct

Sources:

https://jakevdp.github.io/PythonDataScienceHandbook/05.05-naive-bayes.html

https://github.com/benjaminwilson/python-clustering-exercises